In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub
from langchain_upstage import ChatUpstage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore

# 환경변수를 불러옴
load_dotenv()

# Upstage에서 제공하는 Embedding Model을 활용해서 'chunk'를 vector화 
embedding = UpstageEmbeddings(model="embedding-query")
index_name = 'tax-markdown-index'

# 데이터 저장 
database = PineconeVectorStore.from_existing_index(document_list, embedding, index_name=index_name)

llm = ChatUpstage()
prompt = hub.pull("rlm/rag-prompt")
retriever = database.as_retriever(search_kwargs={'k':4})


qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt}
)


# query -> 직장인 -> 거주자 chain 추가
ai_message = qa_chain.invoke({"query": query})




dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

# dictionary_chain을 qa_chain과 연결해주기
tax_chain = {"query": dictionary_chain} | qa_chain

In [16]:
new_question = dictionary_chain.invoke({"question" : query})

In [17]:
query

'연봉 5천만원인 직장인의 종합소득세는 얼마인가요? 알려줘!'

In [18]:
new_question

'연봉 5천만원인 거주자의 종합소득세는 얼마인가요?'

In [19]:
ai_response = tax_chain.invoke({"question" : query})

In [21]:
ai_response

{'query': '연봉 5천만원인 거주자의 종합소득세는 얼마인가요?',
 'result': '5천만원인 경우의 종합소득세는 7,846,000원입니다.'}